### MNIST

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

# move to GPU (if applicable)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
# choose network configuration and dataset
backdoor = Backdoor(VggNet, config='16-layer', channels=1, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.MNIST, ImageEntity)

### Poison setup

In [3]:
# create patches, target labels, and define transformations/merges 
patches = ImagePatch((9, 9), 1, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = ([ExpandTransform(1)], [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
# poison 20% of the data
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
# train the base DNN & the trojan DNN 
loss = backdoor.train(2, epochs=1, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/1 | 24.99% | Loss: 0.2906 | Samples trained: 14992/60000
Epoch 1/1 | 49.97% | Loss: 0.1887 | Samples trained: 29984/60000
Epoch 1/1 | 74.96% | Loss: 0.1485 | Samples trained: 44976/60000
Epoch 1/1 | 99.95% | Loss: 0.1260 | Samples trained: 59968/60000
Epoch 1 complete | Loss: 0.1259
Training complete | Net Average Loss: 0.1259 | Total epochs: 1
Training started
Epoch 1/1 | 24.99% | Loss: 0.5536 | Samples trained: 14992/60000
Epoch 1/1 | 49.97% | Loss: 0.3588 | Samples trained: 29984/60000
Epoch 1/1 | 74.96% | Loss: 0.2826 | Samples trained: 44976/60000
Epoch 1/1 | 99.95% | Loss: 0.2372 | Samples trained: 59968/60000
Epoch 1 complete | Loss: 0.2372
Training complete | Net Average Loss: 0.2372 | Total epochs: 1


### Evaluating Backdoor

In [6]:
# evulate backdoor based on VA & ASR
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 98.89% | Trojan 98.82% | Difference -0.07%
Base Accuracy on Poison 0.64% | Attack Success Rate (ASR): 93.53%
Average Tensor Distance: 247.67 | Net Tensor Difference 14859982.38


### Creating a Defense

In [7]:
# init the defense
defense = Defense(backdoor)

### Detection setup

In [8]:
# get base & trojan DNN, both clean & poisoned entitysets, sizes ranges to create synthetic patch
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [9]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for base DNN
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 98.78% | Original Accuracy 98.89% | Difference -0.11%
Patch size (5, 5) | Synthetic Poison Accuracy 98.22% | Original Accuracy 98.89% | Difference -0.67%
Patch size (7, 7) | Synthetic Poison Accuracy 97.36% | Original Accuracy 98.89% | Difference -1.53%
Patch size (9, 9) | Synthetic Poison Accuracy 96.10% | Original Accuracy 98.89% | Difference -2.79%
Patch size (11, 11) | Synthetic Poison Accuracy 93.33% | Original Accuracy 98.89% | Difference -5.56%
Average Accuracy 96.76% | Average Difference -2.13% | Lowest Score: 93.33% | Likihood of Backdoor: Low


### Detection Trojan

In [10]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for trojan DNN
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 98.70% | Original Accuracy 98.82% | Difference -0.12%
Patch size (5, 5) | Synthetic Poison Accuracy 98.60% | Original Accuracy 98.82% | Difference -0.22%
Patch size (7, 7) | Synthetic Poison Accuracy 92.90% | Original Accuracy 98.82% | Difference -5.92%
Patch size (9, 9) | Synthetic Poison Accuracy 69.73% | Original Accuracy 98.82% | Difference -29.09%
Patch size (11, 11) | Synthetic Poison Accuracy 69.39% | Original Accuracy 98.82% | Difference -29.43%
Average Accuracy 85.86% | Average Difference -12.96% | Lowest Score: 69.39% | Likihood of Backdoor: High


### Blocking Patches setup

In [11]:
# reverse labels (for research VA purposes), create patch to compare variances with in parts of images (same size as patch), define samples to block
labels = {i : i - 1 if i > 0 else 9 for i in range(10)}
patch = ImagePatch((10, 10), 1, 'random')
n = 5000

### Blocking

In [12]:
# block poisoned images with average rgb values the same dimensions as the patch
blockedloader = defense.block(poisontest, patch, labels, n)

### Testing after Block Base

In [13]:
# retest after blocking for base DNN
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
24.84% Testing complete | Loss: 1.2047 | Accuracy: 0.6002
49.68% Testing complete | Loss: 1.2828 | Accuracy: 0.5825
74.52% Testing complete | Loss: 1.2886 | Accuracy: 0.5841
99.36% Testing complete | Loss: 1.2978 | Accuracy: 0.5793
Testing complete | Loss: 1.2981 | Accuracy: 57.96%


### Testing after Block Trojan

In [14]:
# retest after blocking for trojan DNN
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
24.84% Testing complete | Loss: 5.1464 | Accuracy: 0.0449
49.68% Testing complete | Loss: 5.1579 | Accuracy: 0.0405
74.52% Testing complete | Loss: 5.1565 | Accuracy: 0.0422
99.36% Testing complete | Loss: 5.1742 | Accuracy: 0.0423
Testing complete | Loss: 5.1815 | Accuracy: 4.22%


### Defense by retraining

In [15]:
# get a clean dataloader, reset the defense net module, retrain the defense net model (og trojan model)
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=1, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/1 | 24.96% | Loss: 0.0855 | Samples trained: 14976/60000
Epoch 1/1 | 49.92% | Loss: 0.0781 | Samples trained: 29952/60000
Epoch 1/1 | 74.88% | Loss: 0.0766 | Samples trained: 44928/60000
Epoch 1/1 | 99.84% | Loss: 0.0731 | Samples trained: 59904/60000
Epoch 1 complete | Loss: 0.0731
Training complete | Net Average Loss: 0.0731 | Total epochs: 1


### Evaluation after retraining

In [16]:
# re-evaluate the trojan DNN
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 98.89% | Trojan 98.82% | Defense 97.67%
Accuracy on Posion | Base 0.64% | Defense 88.58% | Trojan ASR 93.53%
Difference from Baseline | Trojan -0.07% | Defense -1.22%
Defense Effectiveness | 4.95% decrease in ASR


In [ ]:
defense.view_named_modules()

### Pruning setup

In [18]:
# determine layers to prune, decide percent of neurons to prune, reset the defense (original trojan DNN)
layers = ['layers.28', 'layers.26', 'layers.24', 'layers.21']
amount = 0.5
defense.reset()

### Pruning based on l-Infinity norm of weights from layers

In [19]:
# prune 50% the weights of last 4 convolutional layers based on the l-Infinity norm
defense.prune(layers, amount)

### Evaluation after prune

In [20]:
# evaluate the model after pruning
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 98.89% | Trojan 98.82% | Defense 10.32%
Accuracy on Posion | Base 0.64% | Defense 12.39% | Trojan ASR 93.53%
Difference from Baseline | Trojan -0.07% | Defense -88.57%
Defense Effectiveness | 81.14% decrease in ASR


### Retraining to recover

In [21]:
# retrain to recover
loss = defense.retrain(dataloader, epochs=1, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/1 | 24.96% | Loss: 1.2171 | Samples trained: 14976/60000
Epoch 1/1 | 49.92% | Loss: 1.2169 | Samples trained: 29952/60000
Epoch 1/1 | 74.88% | Loss: 1.2174 | Samples trained: 44928/60000
Epoch 1/1 | 99.84% | Loss: 1.2149 | Samples trained: 59904/60000
Epoch 1 complete | Loss: 1.2149
Training complete | Net Average Loss: 1.2149 | Total epochs: 1


### Evaluation after retraining

In [22]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 98.89% | Trojan 98.82% | Defense 87.67%
Accuracy on Posion | Base 0.64% | Defense 29.75% | Trojan ASR 93.53%
Difference from Baseline | Trojan -0.07% | Defense -11.22%
Defense Effectiveness | 63.78% decrease in ASR
